In [129]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import pandas as pd
import pickle
import random
import tensorflow as tf

In [130]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [131]:
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

43 documents
7 classes ['bud', 'goodbye', 'greeting', 'options', 'rose', 'thanks', 'thorn']
71 unique stemmed words ["'m", "'s", ',', 'anyon', 'ar', 'awesom', 'bad', 'be', 'bet', 'bro', 'bye', 'ca', 'can', 'chat', 'com', 'could', 'day', 'do', 'enough', 'excit', 'for', 'forward', 'fun', 'good', 'goodby', 'happy', 'hard', 'hello', 'help', 'hi', 'hop', 'how', 'i', 'is', 'it', 'lat', 'lik', 'look', 'lov', 'me', 'much', "n't", 'nee', 'next', 'nic', 'not', 'off', 'provid', 'real', 'see', 'smil', 'so', 'soon', 'stop', 'stressful', 'support', 'thank', 'that', 'ther', 'til', 'tim', 'to', 'tomorrow', 'tough', 'up', 'upset', 'wait', 'was', 'what', 'yo', 'you']


In [132]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [133]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [134]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [135]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
43/43 [==============================] - 0s - loss: 1.9543 - acc: 0.1395     
Epoch 2/200
43/43 [==============================] - 0s - loss: 1.9842 - acc: 0.0930     
Epoch 3/200
43/43 [==============================] - 0s - loss: 1.8661 - acc: 0.2791     
Epoch 4/200
43/43 [==============================] - 0s - loss: 1.8394 - acc: 0.3023         
Epoch 5/200
43/43 [==============================] - 0s - loss: 1.7944 - acc: 0.3488     
Epoch 6/200
43/43 [==============================] - 0s - loss: 1.7443 - acc: 0.3953     
Epoch 7/200
43/43 [==============================] - 0s - loss: 1.6275 - acc: 0.4186         
Epoch 8/200
43/43 [==============================] - 0s - loss: 1.6743 - acc: 0.3023         
Epoch 9/200
43/43 [==============================] - 0s - loss: 1.5653 - acc: 0.4884     
Epoch 10/200
43/43 [==============================] - 0s - loss: 1.5002 - acc: 0.4651         
Epoch 11/200
43/43 [==============================] - 0s - loss: 1.3610 - acc: 0.58

43/43 [==============================] - 0s - loss: 0.1235 - acc: 0.9535     
Epoch 91/200
43/43 [==============================] - 0s - loss: 0.0673 - acc: 1.0000     
Epoch 92/200
43/43 [==============================] - 0s - loss: 0.0900 - acc: 0.9767     
Epoch 93/200
43/43 [==============================] - 0s - loss: 0.0713 - acc: 1.0000     
Epoch 94/200
43/43 [==============================] - 0s - loss: 0.1319 - acc: 0.9302     
Epoch 95/200
43/43 [==============================] - 0s - loss: 0.0962 - acc: 0.9535     
Epoch 96/200
43/43 [==============================] - 0s - loss: 0.0655 - acc: 1.0000     
Epoch 97/200
43/43 [==============================] - 0s - loss: 0.0651 - acc: 0.9767         
Epoch 98/200
43/43 [==============================] - 0s - loss: 0.0657 - acc: 0.9767     
Epoch 99/200
43/43 [==============================] - 0s - loss: 0.0754 - acc: 1.0000     
Epoch 100/200
43/43 [==============================] - 0s - loss: 0.0981 - acc: 0.9767     
Epoch 1

43/43 [==============================] - 0s - loss: 0.0710 - acc: 0.9535     
Epoch 179/200
43/43 [==============================] - 0s - loss: 0.0916 - acc: 0.9767     
Epoch 180/200
43/43 [==============================] - 0s - loss: 0.0775 - acc: 0.9535     
Epoch 181/200
43/43 [==============================] - 0s - loss: 0.0711 - acc: 0.9535     
Epoch 182/200
43/43 [==============================] - 0s - loss: 0.0331 - acc: 1.0000     
Epoch 183/200
43/43 [==============================] - 0s - loss: 0.0440 - acc: 0.9767     
Epoch 184/200
43/43 [==============================] - 0s - loss: 0.0340 - acc: 1.0000     
Epoch 185/200
43/43 [==============================] - 0s - loss: 0.0786 - acc: 0.9767         
Epoch 186/200
43/43 [==============================] - 0s - loss: 0.0491 - acc: 0.9535         
Epoch 187/200
43/43 [==============================] - 0s - loss: 0.0432 - acc: 1.0000     
Epoch 188/200
43/43 [==============================] - 0s - loss: 0.1034 - acc: 0.9535

In [136]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [137]:
p = bow("hello how are you", words)
print (p)
print (classes)
x = p.tolist()

model.predict( np.array( [p,] )  )

found in bag: hello
found in bag: how
found in bag: ar
found in bag: you
[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
['bud', 'goodbye', 'greeting', 'options', 'rose', 'thanks', 'thorn']


array([[7.4488192e-08, 3.4603332e-07, 9.9998772e-01, 1.6776564e-06,
        1.0108719e-05, 5.5678706e-09, 2.0941066e-08]], dtype=float32)

In [138]:
def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    p = bow(sentence, words)
    results = model.predict( np.array( [p,] )  )
    arr = np.where(results == np.amax(results))
    print(classes[arr[1][0]])

In [145]:
classify_local()

found in bag: to
found in bag: you
found in bag: lat
goodbye
